# Comparison of GWLinearRegression to MGWR's implementation

Both should yield the same results.

In [1]:
import geopandas as gpd
import pandas as pd
from geodatasets import get_path
from mgwr.gwr import GWR
from numpy.testing import assert_array_almost_equal

from gwlearn.linear_model import GWLinearRegression


In [2]:
gdf = gpd.read_file(get_path("geoda.ncovr"))

In [3]:
gdf.shape

(3085, 70)

In [4]:
# It is in the geographic coords in the  US and we need to work with distances. Re-project and use only points as the graph builder will require points anyway.
gdf = gdf.set_geometry(gdf.representative_point()).to_crs(5070)

In [5]:
y = gdf["FH90"]

## Adaptive bisquare

gwlearn

In [6]:
gwlr = GWLinearRegression(
    bandwidth=250,
    fixed=False,
    n_jobs=-1,
    keep_models=False,
)
gwlr.fit(
    gdf.iloc[:, 9:15],
    y,
    gdf.geometry,
)

mgwr

In [7]:
gwr = GWR(
    coords=gdf.geometry.get_coordinates(),
    y=y.values.reshape(-1, 1),
    X=gdf.iloc[:, 9:15].values,
    bw=250,
    fixed=False,
)
res = gwr.fit()

Local R2

In [8]:
assert_array_almost_equal(gwlr.local_r2_, res.localR2.flatten())

Pred y

In [9]:
assert_array_almost_equal(gwlr.focal_pred_, res.predy.flatten())

TSS

In [10]:
assert_array_almost_equal(gwlr.TSS_, res.TSS.flatten())

RSS

In [11]:
assert_array_almost_equal(gwlr.RSS_, res.RSS.flatten())

## Fixed bisquare

gwlearn

In [12]:
gwlr = GWLinearRegression(
    bandwidth=500_000,
    fixed=True,
    n_jobs=-1,
    keep_models=False,
)
gwlr.fit(
    gdf.iloc[:, 9:15],
    y,
    gdf.geometry,
)

mgwr

In [13]:
gwr = GWR(
    coords=gdf.geometry.get_coordinates(),
    y=y.values.reshape(-1, 1),
    X=gdf.iloc[:, 9:15].values,
    bw=500_000,
    fixed=True,
)
res = gwr.fit()

Local R2

In [14]:
assert_array_almost_equal(gwlr.local_r2_, res.localR2.flatten())

Pred y

In [15]:
assert_array_almost_equal(gwlr.focal_pred_, res.predy.flatten())

TSS

In [16]:
assert_array_almost_equal(gwlr.TSS_, res.TSS.flatten())

RSS

In [17]:
assert_array_almost_equal(gwlr.RSS_, res.RSS.flatten())